# Everybody Codes 2024

## Imports and definitions

In [1]:
import re
from itertools import batched, count, cycle, repeat, combinations, accumulate
from collections import Counter, defaultdict
from statistics import median_low
from math import isqrt, inf


def ecin(day, part):
    with open(f'input/{day}.{part}') as f:
        return f.read().strip()

## [Quest 1: The Battle for the Farmlands](https://everybody.codes/event/2024/quests/1)

In [2]:
def mult_value(*args):
    potions = {'A': 0, 'x': 0, 'B': 1, 'C': 3, 'D': 5}
    bonus = {0: 0, 1: 0, 2: 2, 3: 6}
    values = Counter(args)
    nonempty = len(args) - values['x']
    return bonus[nonempty] + sum(potions[t] * c for t, c in values.items())


A = sum(mult_value(x) for x in ecin(1, 1))
assert A == 1299


A = sum(mult_value(a, b) for a, b in batched(ecin(1, 2), 2))
assert A == 5699


A = sum(mult_value(a, b, c) for a, b, c in batched(ecin(1, 3), 3))
assert A == 28170

## [Quest 2: The Runes of Power](https://everybody.codes/event/2024/quests/2)

In [3]:
def words_regex(words, bidi=False):
    words_ = set(words.split(':')[1].split(','))
    if bidi:
        words_ |= {w[::-1] for w in words_}
    words_ = sorted(words_, reverse=True)
    regex = '(?=(' +  '|'.join(words_) + '))'
    return regex


def transpose_text(text):
    return '\n'.join(
        ''.join(c)
        for c in zip(*text.split('\n'))
    )

words, text = ecin(2, 1).split('\n\n')
A = sum(1 for _ in re.finditer(words_regex(words), text))
assert A == 31


def n_matching_symbols(regex, text):
    symbols = 0
    prev = range(0, 0)
    
    for match in re.finditer(regex, text):
        curr = range(match.start(), match.start() + len(match.group(1)))
        if prev.start <= curr.start <= prev.stop <= curr.stop:
            prev = range(prev.start, curr.stop)
        elif prev.start <= prev.stop < curr.start <= curr.stop:
            symbols += prev.stop - prev.start
            prev = curr
    
    return symbols + (prev.stop - prev.start)

words, text = ecin(2, 2).split('\n\n')
A = n_matching_symbols(words_regex(words, bidi=True), text)
assert A == 5345


def n_matching_symbols_grid(regex, text):
    matching = set()

    for nline, line in enumerate(text.split('\n')):
        for match in re.finditer(regex, line + line):
            if match.start() < len(line):
                matching |= {
                    (nline, t % len(line)) 
                    for t in range(match.start(), match.start() + len(match.group(1)))
                }
            else: break

    for nline, line in enumerate(transpose_text(text).split('\n')):
        for match in re.finditer(regex, line):
            matching |= {
                (t, nline) 
                for t in range(match.start(), match.start() + len(match.group(1)))
            }

    return len(matching)


words, text = ecin(2, 3).split('\n\n')
A = n_matching_symbols_grid(words_regex(words, bidi=True), text)
assert A == 11884

## [Quest 3: Mining Maestro](https://everybody.codes/event/2024/quests/3)

In [4]:
def read_coord(text, symbol='#'):
    for i, row in enumerate(text.split('\n')):
        for j, char in enumerate(row):
            if char == symbol:
                yield i, j


def orthogonally_adjacent(i, j):
    yield from ((i+1, j), (i-1, j), (i, j+1), (i, j-1))


def diagonally_adjacent(i, j):
    yield from (
        (i-1, j-1), (i-1, j), (i-1, j+1),
        (i  , j-1),           (i  , j+1),
        (i+1, j-1), (i+1, j), (i+1, j+1)
    )

def max_mining(coords, adjacency=orthogonally_adjacent):
    if not (c := set(coords)): return 0
    return len(c) + max_mining({
        (i, j)
        for (i, j) in c
        if all(u in c for u in adjacency(i, j))
    }, adjacency=adjacency)


A = max_mining(read_coord(ecin(3, 1)))
assert A == 116


A = max_mining(read_coord(ecin(3, 2)))
assert A == 2678


text = ecin(3, 3)
rows = text.split('\n')
height, width = len(rows), len(rows[0])
A = max_mining(read_coord(text), diagonally_adjacent)
assert A == 10698

## [Quest 4: Royal Smith's Puzzle](https://everybody.codes/event/2024/quests/4)

In [5]:
nails = [*map(int, ecin(4, 1).split('\n'))]
A = sum(nails) - len(nails) * min(nails)
assert A == 87


nails = [*map(int, ecin(4, 2).split('\n'))]
A = sum(nails) - len(nails) * min(nails)
assert A == 894606


nails = [*map(int, ecin(4, 3).split('\n'))]
target = median_low(nails)
A = sum(abs(x - target) for x in nails)
assert A == 120168130

## [Quest 5: Pseudo-Random Clap Dance](https://everybody.codes/event/2024/quests/5)

In [6]:
def pivoted_input(text):
    return [
        list(t)
        for t in zip(*[
            [int(u) for u in x.split()]
            for x in text.split('\n')
        ])
    ]


def first_elements(lists):
    return int(''.join(str(l[0]) for l in lists))


def move(lists, r):
    clapper_column = r % len(lists)
    clapper = lists[clapper_column].pop(0)
    target_col = lists[(clapper_column+1) % len(lists)]
    position = (clapper - 1) % (2 * len(target_col))
    if position >= len(target_col):
        position = 2 * len(target_col) - position
    target_col.insert(position, clapper)


def move_n(lists, n):
    for i in range(n):
        move(lists, i)


lists = pivoted_input(ecin(5, 1))
move_n(lists, 10)
A = first_elements(lists)
assert A == 3222


def repeat_until_n(lists, n):
    c = Counter()
    for i in count(1):
        move(lists, i-1)
        t = first_elements(lists)
        c[t] += 1
        if c[t] == n: return i, t


rnd, elements = repeat_until_n(pivoted_input(ecin(5, 2)), 2024)
A = rnd * elements
assert A == 12881059969561


def max_repeat_forever(lists):
    max_val = 0
    seen = set()
    for i in count(1):
        move(lists, i-1)
        if str(lists) in seen: break
        seen.add(str(lists))
        max_val = max(max_val, first_elements(lists))
    return max_val

A = max_repeat_forever(pivoted_input(ecin(5, 3)))
assert A == 9107100310021002

## [Quest 6: The Tree of Titans](https://everybody.codes/event/2024/quests/6)

In [7]:
def to_adjacency_list(text):
    res = defaultdict(list)
    for line in text.split('\n'):
        k, v = line.split(':')
        vs = v.split(',')
        res[k] = vs
    return res


def path_lengths(tree):
    Q = [('RR', [])]
    seen = set()
    paths_by_len = defaultdict(list)
    while Q:
        node, path = Q.pop()
        seen.add(node)
        for adj in tree[node]:
            if adj == '@':
                paths_by_len[len(path)].append(path + [node, adj])
            elif adj not in seen:
                Q.append((adj, path + [node]))
    return paths_by_len


def unique_len(tree):
    pl = path_lengths(tree)
    for paths in pl.values():
        if len(paths) == 1:
            return paths[0]
       
       
A = ''.join(unique_len(to_adjacency_list(ecin(6, 1))))
assert A == 'RRJQRMCNQMSP@'


A = ''.join(s[0] for s in unique_len(to_adjacency_list(ecin(6, 2))))
assert A == 'RBPCLWTQKZ@'


A = ''.join(s[0] for s in unique_len(to_adjacency_list(ecin(6, 3))))
assert A == 'RKBXWHVBZGNF@'

## [Quest 7: Not Fast but Furious](https://everybody.codes/event/2024/quests/7)

In [8]:
def to_actions_dict(text):
    actions = {}
    for line in text.split('\n'):
        name, act = line.split(':')
        acts = act.split(',')
        actions[name] = acts
    return actions

def to_racetrack(text):
    grid = [[*line] for line in text.split('\n')]
    height, width = len(grid), len(grid[0])
    for line in grid:
        if (d := width - len(line)) > 0:
            line.extend([' '] * d)
    x, y = 0, 1
    seen = set()
    while True:
        yield grid[x][y]
        seen.add((x, y))
        for xx, yy in ((x, y+1), (x+1, y), (x, y-1), (x-1, y)):
            if (
                0 <= xx < height
                and 0 <= yy < width
                and (xx, yy) not in seen
                and grid[xx][yy] != ' '
            ):
                x, y = xx, yy
                break
        else: break

def total_power(actions, racetrack=repeat('=')):
    curr = 10
    tot = 0
    for action, tile in zip(actions, racetrack):
        match action, tile:
            case  _ , '+': curr += 1
            case  _ , '-': curr = max(0, curr-1)
            case '+',  _ : curr += 1
            case '-',  _ : curr = max(0, curr-1)
            case  _ ,  _ : pass
        tot += curr
    return tot


actions_dict = to_actions_dict(ecin(7, 1))
A = ''.join(n for _, n in sorted(
    (
        (total_power(actions), name) 
        for name, actions in actions_dict.items()
    ), reverse=True
))
assert A == 'BAHIFJEGC'


actions, racetrack = ecin(7, 2).split('\n\n')
actions_dict = to_actions_dict(actions)
racetrack_lst = list(to_racetrack(racetrack))
A = ''.join(n for _, n in sorted(
    (
        (total_power(cycle(actions), racetrack_lst * 10), name)
        for name, actions in actions_dict.items()
    ), reverse=True
))
assert A == 'KCEDFJIAB'


def total_power_symbolic(actions, racetrack):
    curr = Counter({'': 10})
    tot = Counter()
    for action, tile in zip(actions, racetrack):
        match action, tile:
            case  _ , '+': curr += Counter({'': 1})
            case  _ , '-': curr -= Counter({'': 1})
            case  _ ,  _ : curr += action
        tot += curr
    return tot


def combinations_gt(symbols, symbolic_power, target):
    for plus in combinations(symbols, 5):
        for minus in combinations((symbols - {*plus}), 3):
            if (
                    symbolic_power['']
                    + sum(symbolic_power[p] for p in plus)
                    - sum(symbolic_power[m] for m in minus)
            ) > target:
                yield plus, minus


opponent, racetrack = ecin(7, 3).split('\n\n')
opponent_actions = to_actions_dict(opponent)['A']
racetrack_lst = list(to_racetrack(racetrack))
racetrack_loops = racetrack_lst * 2024
symbols = 'abcdefghijk'

opponent_power = total_power(cycle(opponent_actions), racetrack_loops)
our_power = total_power_symbolic(cycle((Counter(j) for j in symbols)), racetrack_loops)
A = sum(1 for _ in combinations_gt(set(symbols), our_power, opponent_power))
assert A == 6103

## [Quest 8: A Shrine for Nullpointer](https://everybody.codes/event/2024/quests/8)

In [9]:
N = int(ecin(8, 1))
A = (isqrt(N) * 2 + 1) * ((isqrt(N) + 1) ** 2 - N)
assert A == 9681159

N, M, blocks = map(int, ecin(8, 2).split('\n'))

def missing_blocks(N, M, blocks):
    for i in count():
        blocks -= pow(N, i, mod=M) * (2 * i + 1)
        if blocks < 0: break
    return i, -blocks


rnd, missing = missing_blocks(N, M, blocks)
A = (2 * rnd + 1) * missing
assert A == 107695974


def missing_blocks_3(N, M, blocks):
    columns = [1]
    for i in count(1):
        columns.append(((columns[-1] * N) % M) + M)
        heights = list(accumulate(columns[::-1]))
        heights += heights[-2::-1]
        used_blocks = heights[0] + heights[-1] + sum(
            j - (N * (2 * i + 1) * j) % M for j in heights[1:-1]
        )
        if used_blocks >= blocks:
            return used_blocks - blocks


N, M, blocks = map(int, ecin(8, 3).split('\n'))
A = missing_blocks_3(N, M, blocks)
assert A == 41067

## [Quest 9: Sparkling Bugs](https://everybody.codes/event/2024/quests/9)

In [10]:
def count_ways(n, options):
    D = [0] + [inf for _ in range(n)]
    for i in range(n+1):
        D[i] = 1 + min((D[i-j] for j in options if j <= i), default=D[i]-1)
    return D

ns = [*map(int, ecin(9, 1).split('\n'))]
D = count_ways(max(ns), (1, 3, 5, 10))
A = sum(D[i] for i in ns)
assert A == 13612


ns = [*map(int, ecin(9, 2).split('\n'))]
D = count_ways(max(ns), (1, 3, 5, 10, 15, 16, 20, 24, 25, 30))
A = sum(D[i] for i in ns)
assert A == 5072


def count_ways_split(n, D):
    lo = (n + 1) // 2 - 50
    hi = n // 2 + 50
    return min(D[i] + D[n-i] for i in range(lo, hi+1))


ns = [*map(int, ecin(9, 3).split('\n'))]
D = count_ways(max(ns), (
    1, 3, 5, 10, 15, 16, 20, 24, 25, 30,
    37, 38, 49, 50, 74, 75, 100, 101
))
A = sum(count_ways_split(n, D) for n in ns)
assert A == 154231